## 라이브러리 로드

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져옵니다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱합니다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러옵니다.
import pandas as pd

## 서울시 코로나19 발생동향
* [코로나19](http://www.seoul.go.kr/coronaV/coronaStatus.do)

## requests 로 html 문서 받아오기

In [2]:
# 크롤링 할 사이트
base_url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
response = requests.get( base_url )
response

<Response [200]>

In [3]:
if response.status_code == 200:
    soup = bs(response.text, 'html.parser')

str(soup)[:1000]

'<!-- ######################################################################## -->\n<!-- \r\n\tmenu name:\t\t코로나 바이러스 현황 단독 페이지\r\n-->\n<!-- ######################################################################## -->\n<!DOCTYPE html>\n\n<html lang="ko">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport">\n<title>코로나19</title>\n<meta content="코로나19" name="description"/>\n<meta content="website" property="og:type"/>\n<meta content="코로나19 - 2주간의 \'잠시 멈춤\' 캠페인" property="og:title"/>\n<meta content="코로나19를 멈추기 위해 우리도 잠시 멈춰요." property="og:description"/>\n<meta content="https://www.seoul.go.kr/coronaV/coronaStatus.do" property="og:url"/>\n<meta content="http://www.seoul.go.kr/res_newseoul/images/corona/pic_facebook_20200304.jpg" property="og:image"/>\n<meta content="summary" name="twitter:card"/>\n<meta content="코로나19 - 2주간의 \'잠시 멈춤\' 캠페인" name="twitter:title"/>\n<meta content="코로나19

## 서울 확진자 현황

In [4]:
#move-cont1 > div:nth-child(2) > table
#move-cont1 > div:nth-child(2) > table.tstyle05.tstyleP > thead > tr > th:nth-child(1)

table = soup.select("#move-cont1 > div > table.tstyle05.tstyleP")
# table

In [5]:
trs =  table[0].select("tr")
# trs

In [12]:
table = []
for tr in trs:
    tds = tr.select("td")
    if len(tds) > 0 :
        row = []
        for td in tds:
            val = td.get_text()
            row.append(val)
        table.append(row)
print(table[:5])

[['확인중', '3.7.', "남('64)", '강남구', '-', '타시도 확진자 접촉자', '서울의료원'], ['확인중', '3.7.', "남('29)", '고양', '-', '타시도 확진자 접촉자', '서울의료원'], ['확인중', '3.7.', "남('13)", '관악구', '-', '타시도 확진자 접촉자', '보라매병원'], ['확인중', '3.7.', "남('66)", '양천구', '-', '서울 114번 접촉자', '서남병원'], ['확인중', '3.7.', "남('55)", '서초구', '-', '#935 접촉자', '서남병원']]


In [7]:
# 컬럼명 만들기
cols = trs[0].get_text()
cols = cols.split("\n")[1:-1]
cols

['연번', '환자', '확진일', '성별(출생년)', '거주지', '여행력', '접촉력', '조치사항']

In [8]:
df = pd.DataFrame(table, columns=cols[1:])
df.head()

,환자,확진일,성별(출생년),거주지,여행력,접촉력,조치사항
0,확인중,3.7.,남('64),강남구,-,타시도 확진자 접촉자,서울의료원
1,확인중,3.7.,남('29),고양,-,타시도 확진자 접촉자,서울의료원
2,확인중,3.7.,남('13),관악구,-,타시도 확진자 접촉자,보라매병원
3,확인중,3.7.,남('66),양천구,-,서울 114번 접촉자,서남병원
4,확인중,3.7.,남('55),서초구,-,#935 접촉자,서남병원


In [9]:
df = df.sort_values(by="확진일", ascending=False)
df.to_csv("covid-19-seoul.csv", index=False)

In [10]:
pd.read_csv("covid-19-seoul.csv").head(10)

,환자,확진일,성별(출생년),거주지,여행력,접촉력,조치사항
0,확인중,3.7.,남('64),강남구,-,타시도 확진자 접촉자,서울의료원
1,확인중,3.7.,남('13),관악구,-,타시도 확진자 접촉자,보라매병원
2,확인중,3.7.,남('66),양천구,-,서울 114번 접촉자,서남병원
3,확인중,3.7.,남('55),서초구,-,#935 접촉자,서남병원
4,확인중,3.7.,남('87),은평구,"스페인, 프랑스",확인중,서북병원
5,확인중,3.7.,남('29),고양,-,타시도 확진자 접촉자,서울의료원
6,확인중,3.6.,남('81),성북구,-,-,서남병원
7,확인중,3.6.,여('69),양천구,-,타시도 확진자 접촉,보라매병원
8,확인중,3.6.,여('56),강북구,-,서울 104번 접촉,서남병원
9,확인중,3.6.,남('55),강북구,-,서울 104번 접촉,서남병원


In [11]:
df["접촉력"].value_counts()

확인중              15
성동구 주상복합 관련      13
해외 접촉            13
타시도 확진자 접촉        8
은평성모병원관련          7
은평성모병원 관련         6
대구 방문             5
타시도 확진자 접촉자       3
타시도 확진자 접촉        3
대구 확진자 접촉         3
#6 접촉자            3
신천지대구교회 관련        2
명성교회 관련           2
#794 접촉자          2
-                 2
#780 접촉자          2
서울 104번 접촉        2
#83접촉자            1
#2621번 접촉자        1
여의도 건설현장관련        1
서울 114번 접촉자       1
 타시도 확진자 접촉       1
경찰병원관련            1
대구 방문(명성교회)       1
부동산 중개 법인관련       1
여의도 건설현장 관련       1
#3 접촉자            1
해외 접촉(추정)         1
#30접촉자(추정)        1
#29접촉자            1
#1247 접촉자         1
대구 거주자            1
#935 접촉자          1
#136접촉자           1
부동산 중개업 관련        1
은평성모병원 연관성 검토     1
확잍중               1
#5 접촉자            1
#780 가족(아들)       1
서울재생병원 관련         1
#6접촉자(추정)         1
쇼핑몰 관련            1
#56접촉자            1
부동산중개법인관련         1
해외접촉              1
Name: 접촉력, dtype: int64